<a href="https://colab.research.google.com/github/NirantK/span_cat_demo/blob/main/spaCy_TFM_CoNLL2003.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content/
!git clone https://github.com/explosion/spacy
%cd /content/
!git clone https://github.com/NirantK/span_cat_demo

/content
fatal: destination path 'spacy' already exists and is not an empty directory.
/content
fatal: destination path 'span_cat_demo' already exists and is not an empty directory.


In [2]:
%cd /content/spacy
!git checkout feature/span-categorizer-v3
!pip install spacy-transformers
!pip install -r requirements.txt
!pip install --no-build-isolation --editable .

/content/spacy
Already on 'feature/span-categorizer-v3'
Your branch is up to date with 'origin/feature/span-categorizer-v3'.
Obtaining file:///content/spacy
    Preparing wheel metadata ... done
  Found existing installation: spacy 3.1.0
    Can't uninstall 'spacy'. No files were found to uninstall.
  Running setup.py develop for spacy


In [3]:
%cd /content/span_cat_demo
!git pull -a
%cd /content/span_cat_demo/conll2003
!python -m spacy project assets
!python -m spacy project run corpus
# !python -m spacy project run train
# !python -m spacy project run evaluate

/content/span_cat_demo
Already up to date.
/content/span_cat_demo/conll2003
2021-06-23 09:35:26.606257: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
ℹ Fetching 3 asset(s)
✔ Downloaded asset
/content/span_cat_demo/conll2003/assets/conll2003/dev.iob
✔ Downloaded asset
/content/span_cat_demo/conll2003/assets/conll2003/train.iob
✔ Downloaded asset
/content/span_cat_demo/conll2003/assets/conll2003/test.iob
2021-06-23 09:35:30.678873: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0

=================================== corpus ===================================
ℹ Skipping 'corpus': nothing changed


In [64]:
from spacy.tokens import DocBin, Doc
# use the gold labels to find the upper bound for recall for any suggester
# i have the gold labels from corpus for dev, and test both
# i need the suggester output in some way
from spacy.util import registry
from typing import List, Callable, Optional
from thinc.api import Ops, get_current_ops, to_numpy
from thinc.types import Ragged
from pathlib import Path
import numpy as np

def build_ngram_suggester(sizes: List[int]) -> Callable[[List[Doc]], Ragged]:
    """Suggest all spans of the given lengths. Spans are returned as a ragged
    array of integers. The array has two columns, indicating the start and end
    position."""

    def ngram_suggester(docs: List[Doc], *, ops: Optional[Ops] = None) -> Ragged:
        if ops is None:
            ops = get_current_ops()
        spans = []
        lengths = []
        for doc in docs:
            starts = ops.xp.arange(len(doc), dtype="i")
            starts = starts.reshape((-1, 1))
            length = 0
            for size in sizes:
                if size <= len(doc):
                    starts_size = starts[:len(doc) - (size - 1)]
                    spans.append(ops.xp.hstack((starts_size, starts_size + size)))
                    length += spans[-1].shape[0]
                if spans:
                    assert spans[-1].ndim == 2, spans[-1].shape
            lengths.append(length)
        if len(spans) > 0:
            output = Ragged(ops.xp.vstack(spans), ops.asarray(lengths, dtype="i"))
        else:
            output = Ragged(ops.xp.zeros((0,0)), ops.asarray(lengths, dtype="i"))

        assert output.dataXd.ndim == 2
        return output

    return ngram_suggester

n_gram_suggester = build_ngram_suggester(sizes = [1, 2, 3])

In [10]:
import spacy
from spacy.tokens import DocBin
nlp = spacy.blank("en")

In [6]:
loc = Path("/content/span_cat_demo/conll2003/corpus/test.spacy")
assert loc.exists()

In [11]:
doc_bin = DocBin().from_disk(loc)
docs = list(doc_bin.get_docs(nlp.vocab))

In [108]:
def get_gold_spans(docs: List[Doc])->np.array:
    doc_ent_pos, lengths = [], []
    for doc in docs:
        ent_pos = [[ent.start, ent.end] for ent in doc.ents]
        doc_ent_pos.extend(ent_pos)
        lengths.append(len(ent_pos))
    return np.array(doc_ent_pos), np.array(lengths)

gold_spans, lengths = get_gold_spans(docs)
ngram_pos_list = n_gram_suggester(docs)

In [154]:
assert ngram_pos_list.lengths.shape == lengths.shape
ngram_pos_list.data.shape, gold_spans.shape, lengths.shape, ngram_pos_list.lengths.shape
# type(gold_spans[0])

((137784, 2), (5648, 2), (369,), (369,))

In [155]:
suggest_start, ent_start = 0, 0
doc_overlaps = []
for suggest_cnt, ent_cnt in zip(ngram_pos_list.lengths, lengths):
    suggestions = ngram_pos_list.data[suggest_start: suggest_start+suggest_cnt]
    suggest_start += suggest_cnt
    entities = gold_spans[ent_start: ent_start+ent_cnt]
    ent_start += ent_cnt
    assert len(entities) ==  ent_cnt
    assert len(suggestions) == suggest_cnt
    # print(len(suggestions), len(entities))
    A, B = entities, suggestions
    # We can create a boolean mask using broadcasting which can be then used to filter the rows in array A which are also present in array B
    # mask = (A[:, None] == B).all(-1).any(1)
    # overlap = A[mask]
    # Alternatively, we can use native Python set tuples
    aset = set([tuple(x) for x in A])
    bset = set([tuple(x) for x in B])
    # print(len(aset), len(bset), aset, bset)
    overlap = aset & bset
    # overlap - np.array([x for x in aset & bset])
    # assert len(overlap) < len(suggestions)
    # assert len(overlap) < len(entities)
    doc_overlaps.append({"intersect": len(overlap), "doc_r": len(overlap) / ent_cnt, "ent_cnt": ent_cnt})

len(doc_overlaps)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:23: RuntimeWarning: invalid value encountered in long_scalars


369

In [158]:
total_ents_found, avg_recall = 0, 0.0
for element in doc_overlaps:
    total_ents_found += element["intersect"]
    if element["ent_cnt"] > 0:
        avg_recall += element["doc_r"]

print(f"Global Recall (Calculated as Total Suggestions/Total Entities): {total_ents_found/gold_spans.shape[0]:.4f}")
print(f"Average (Calculated on a Per Doc Level): {avg_recall/len(doc_overlaps):.4f}")

Global Recall (Calculated as Total Suggestions/Total Entities): 0.9860
Average (Calculated on a Per Doc Level): 0.9731
